In [28]:

# Data treatment

import numpy as np # linear algebra
import polars as pl # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd

# Deep Learning

import torch                  
import torch.nn as nn          
import torch.nn.functional as F  
import torch.optim as optim  
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset, TensorDataset

# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [29]:
df_data = pl.read_csv(r'C:\Users\Lucas\Documents\GitHub\Recommind\data\processed\books_data.csv')
df_ratings = pl.read_csv(r'C:\Users\Lucas\Documents\GitHub\Recommind\data\processed\Books_rating.csv')

In [30]:
df_data = df_data[['Title', 'authors', 'categories', 'ratingsCount']]

In [31]:
df_data = df_data.with_columns(
    pl.col("ratingsCount").fill_null(0),
    pl.col("categories").fill_null("No Category")
)


In [32]:
df_ratings2 = df_ratings[['Id','Title', 'User_id', 'review/score']]

In [33]:
df_ratings2 = df_ratings2.drop_nulls()

In [34]:
df_merged = df_ratings2.join(
    df_data.select(["Title", "authors", "categories", "ratingsCount"]),
    on='Title',
    how='left'
)

In [35]:
import polars as pl
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder
import numpy as np

# 0. Preprocessing the strings

df_merged = df_merged.with_columns(  

    pl.col("categories").str.replace_all('[','',literal=True).str.replace_all(']','',literal=True),
    pl.col("authors").str.replace_all('[','',literal=True).str.replace_all(']','',literal=True),

)


# 1. Ordinal Encoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoded = encoder.fit_transform(df_merged.select(['User_id', 'Id']).to_numpy())

df_encoded = df_merged.with_columns([
    pl.Series(name='User_id', values=encoded[:, 0].astype(int)),
    pl.Series(name='Id', values=encoded[:, 1].astype(int)),
])

df_encoded = df_encoded.filter(
    (pl.col('User_id') != -1) & (pl.col('Id') != -1)
)

n_users = df_encoded.select(pl.col('User_id').max()).item() + 1
n_items = df_encoded.select(pl.col('Id').max()).item() + 1

print(f"n_users = {n_users}, n_items = {n_items}")


# 2. StandardScaler
#scaler = StandardScaler()

#scores = df_encoded.select('review/score').to_numpy()

#scores_scaled = scaler.fit_transform(scores)

#df_encoded = df_encoded.with_columns(
#    pl.Series(name="review/score_scaled", values=scores_scaled.flatten())
#)

# 3. LabelEncoder


authors = df_encoded["authors"].to_numpy()
categories = df_encoded["categories"].to_numpy()

enc_authors = LabelEncoder()
enc_categories = LabelEncoder()

authors_encoded = enc_authors.fit_transform(authors)
categories_encoded = enc_categories.fit_transform(categories)

# Se quiser colocar de volta no DataFrame
df_encoded = df_encoded.with_columns([
    pl.Series("authors", authors_encoded),
    pl.Series("categories", categories_encoded)
])




n_users = 1008961, n_items = 216014


In [36]:
df_encodeds = df_encoded.to_pandas()

## Research

In [37]:
df_encodeds

,Id,Title,User_id,review/score,authors,categories,ratingsCount
0,127550,Its Only Art If Its Well Hung!,974357,4.0,64441,3348,0.0
1,72368,Dr. Seuss: American Icon,538929,5.0,93819,1887,0.0
2,72368,Dr. Seuss: American Icon,758564,5.0,93819,1887,0.0
3,72368,Dr. Seuss: American Icon,436241,4.0,93819,1887,0.0
4,72368,Dr. Seuss: American Icon,288503,4.0,93819,1887,0.0
...,...,...,...,...,...,...,...
2438013,205239,The Idea of History,875975,5.0,94976,6283,3.0
2438014,205239,The Idea of History,923008,4.0,94976,6283,3.0
2438015,205239,The Idea of History,212393,4.0,94976,6283,3.0
2438016,205239,The Idea of History,346325,4.0,94976,6283,3.0


To test the model working on an application, we are going to recommend books for a random user.


In [38]:
user = df_encodeds[df_encodeds['User_id'] == 165515]

In [39]:
user

,Id,Title,User_id,review/score,authors,categories,ratingsCount
2055,160141,1001 Christmas Facts and Fancies,165515,5.0,3728,10668,0.0
4309,189137,The Gods of Mars,165515,5.0,31720,5287,26.0
6386,211531,Foundation,165515,5.0,64868,4635,0.0
8050,47893,King Rat,165515,5.0,51425,5287,19.0
8551,211532,Foundation,165515,5.0,64868,4635,0.0
...,...,...,...,...,...,...,...
2425329,24962,The Wicked Flea (Dog Lover's Mysteries),165515,5.0,111786,5287,0.0
2426095,209299,FOUNDATION,165515,5.0,64868,4635,0.0
2433432,181488,Dandelion Wine,165515,5.0,96234,5287,2.0
2433655,188570,Hemingway: The Paris Years,165515,5.0,83137,1887,4.0


In [40]:
user_items = user['Id']


In [41]:
all_items = df_encodeds['Id'].unique()

In [42]:
items_to_predict = list(set(all_items) - set(user_items))

In [43]:
items_to_predict = pd.DataFrame(items_to_predict, columns=['Id'])

Using the items_to_predict we will create a dataframe to predict the ratings of this books to our user

The correct thing to do is to use the Database and merge the book data and use this merge in the model, but here we will create it using only pandas


In [44]:
books_data = df_encodeds.drop_duplicates(subset='Id')

In [45]:
resultado = pd.merge(books_data, items_to_predict, on='Id', how='inner')

In [46]:
resultado

,Id,Title,User_id,review/score,authors,categories,ratingsCount
0,127550,Its Only Art If Its Well Hung!,974357,4.0,64441,3348,0.0
1,72368,Dr. Seuss: American Icon,538929,5.0,93819,1887,0.0
2,72745,Wonderful Worship in Smaller Churches,1001608,5.0,26068,9198,0.0
3,37578,Whispers of the Wicked Saints,725367,1.0,118279,5287,0.0
4,11808,"Nation Dance: Religion, Identity and Cultural ...",914561,5.0,32403,10668,0.0
...,...,...,...,...,...,...,...
215232,167056,"Old Quebec,: The fortress of New France,",555120,3.0,42066,10668,0.0
215233,53403,On the Life of Christ: Chanted Sermons by the ...,574109,5.0,104868,9198,0.0
215234,205200,Ghost Story,248963,5.0,93247,5287,0.0
215235,11892,An End to Welfare Rights: The Rediscovery of I...,307890,4.0,25003,8775,0.0


In [47]:
resultado = resultado.drop('review/score', axis=1)
resultado['User_id'] = 165515

In [48]:
resultado

,Id,Title,User_id,authors,categories,ratingsCount
0,127550,Its Only Art If Its Well Hung!,165515,64441,3348,0.0
1,72368,Dr. Seuss: American Icon,165515,93819,1887,0.0
2,72745,Wonderful Worship in Smaller Churches,165515,26068,9198,0.0
3,37578,Whispers of the Wicked Saints,165515,118279,5287,0.0
4,11808,"Nation Dance: Religion, Identity and Cultural ...",165515,32403,10668,0.0
...,...,...,...,...,...,...
215232,167056,"Old Quebec,: The fortress of New France,",165515,42066,10668,0.0
215233,53403,On the Life of Christ: Chanted Sermons by the ...,165515,104868,9198,0.0
215234,205200,Ghost Story,165515,93247,5287,0.0
215235,11892,An End to Welfare Rights: The Rediscovery of I...,165515,25003,8775,0.0


In [49]:
class PredictDataset(Dataset):
    def __init__(self, dataframe):
        self.X = dataframe
        self.X = self.X.drop('Title', axis=1).to_numpy()

        self.X = torch.tensor(self.X, dtype=torch.long)

    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx]

dataset = PredictDataset(resultado)
loader = DataLoader(dataset, batch_size=2048, shuffle=True, drop_last=True)



In [50]:
import torch
import torch.nn as nn

class NeuMF(nn.Module):
    def __init__(self, n_users, n_items,n_genders, n_authors, n_factors=8):
        super().__init__()

        # Embeddings for GMF (Generalized Matrix Factorization) path
        self.user_gmf = nn.Embedding(n_users, n_factors)
        self.item_gmf = nn.Embedding(n_items, n_factors)

        # Embeddings for MLP path
        self.user_mlp = nn.Embedding(n_users, n_factors)
        self.item_mlp = nn.Embedding(n_items, n_factors)
        self.item_gender_emb = nn.Embedding(n_genders, n_factors)
        self.item_authors_emb = nn.Embedding(n_authors, n_factors)

        
        # Initialize embeddings with small uniform values
        self.user_gmf.weight.data.uniform_(0, 0.05)
        self.item_gmf.weight.data.uniform_(0, 0.05)
        self.user_mlp.weight.data.uniform_(0, 0.05)
        self.item_mlp.weight.data.uniform_(0, 0.05)

        # MLP input: user + item embedding + text embedding
        input_dim = n_factors * 4 + 1 

        # MLP: several layers with ReLU and Dropout to prevent overfitting
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(256, 32)
        )

        # Final prediction layer: GMF + MLP outputs concatenated
        self.final_layer = nn.Linear(n_factors + 32, 1)

    def forward(self, data):
        users = data[:, 1]
        items = data[:, 0]
        genders = data[:, 3]
        authors = data[:, 2]
        ratingsCount = data[:, 4]



        # GMF path
        gmf_user = self.user_gmf(users)
        gmf_item = self.item_gmf(items)
        gmf_out = gmf_user * gmf_item  # element-wise product

        # Gender and authors 
        gender_emb = self.item_gender_emb(genders)
        authors_emb = self.item_authors_emb(authors)


        # MLP path
        mlp_user = self.user_mlp(users)
        mlp_item = self.item_mlp(items)
        mlp_items = torch.cat([mlp_item, gender_emb, authors_emb,ratingsCount.unsqueeze(1)], dim=1)

        # Concatenate user, item, and text embeddings
        mlp_input = torch.cat([mlp_user, mlp_items], dim=1)
       # print(mlp_input.shape)
        mlp_out = self.mlp(mlp_input)

        # Combine GMF and MLP paths and make final prediction
        final_input = torch.cat([gmf_out, mlp_out], dim=1)
        out = self.final_layer(final_input).squeeze(1)

        return out

In [51]:
import os 
recommind_model = torch.load(os.path.join(r"C:\Users\Lucas\Documents\GitHub\Recommind\models\ncf_model", 'recommind_model.pth'))
model = NeuMF(**recommind_model['config'])
model.load_state_dict(recommind_model['model_state_dict'])


<All keys matched successfully>

In [52]:
model.to(device='cuda')

NeuMF(
  (user_gmf): Embedding(1008961, 16)
  (item_gmf): Embedding(216014, 16)
  (user_mlp): Embedding(1008961, 16)
  (item_mlp): Embedding(216014, 16)
  (item_gender_emb): Embedding(10669, 16)
  (item_authors_emb): Embedding(124098, 16)
  (mlp): Sequential(
    (0): Linear(in_features=65, out_features=1024, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.4, inplace=False)
    (9): Linear(in_features=256, out_features=32, bias=True)
  )
  (final_layer): Linear(in_features=48, out_features=1, bias=True)
)

In [ ]:
model.eval()
number_batch = 0
scores = []
with torch.no_grad():
    for X in loader:
        X = X.to(device)
        outputs = model(X)
        scores.append(outputs.cpu())



In [56]:
scores = scores.flatten()
scores.shape

AttributeError: 'list' object has no attribute 'flatten'

In [ ]:
scores = torch.tensor(scores)

In [ ]:
top_k = 10
top_indices = torch.topk(scores, top_k).indices

In [ ]:
top_items = [items_to_predict.iloc[int(i)] for i in top_indices]

In [60]:

book_df = df_encodeds[['Id', 'Title']]

book_dict = dict(zip(book_df['Id'], book_df['Title']))

In [ ]:
for idx in range(len(top_items)):
        book_id = top_items[idx]['Id']
        book = book_dict[book_id]
        print(f" Book: {book}")

 Book: Lex & Yacc
 Book: Schelers Critique Of Kants Ethics: Continental Thought Series, V. 22 (Series In Continental Thought)
 Book: The Notebook of Elbert Hubbard
 Book: Beggars Ride
 Book: After the Greening: The Browning of Australia
 Book: The Valley of Death: Sergeant Jack Crossman and the Battle of Balaclava
 Book: Changing Habits
 Book: The Dying of the Light
 Book: Test Yourself MCSE Windows 2000 Directory Services Administration (Exam 70-217)
 Book: NEW YORK METS ENCYCLOPEDIA


## Functions

In [71]:
items_to_predict.dtypes

Id    int32
dtype: object

In [80]:
def reco_user(user):
    user_dataframe = df_encodeds[df_encodeds['User_id'] == user]
    user_items = user_dataframe['Id']

    all_items = df_encodeds['Id'].unique()
    items_to_predict = list(set(all_items) - set(user_items))
    items_to_predict = pd.DataFrame(items_to_predict, columns=['Id'])
    books_data = df_encodeds.drop_duplicates(subset='Id')
    result = pd.merge(books_data, items_to_predict, on='Id', how='inner')
    result = result.drop('review/score', axis=1)
    print(result)
    result['User_id'] = user

    return result

def pred_user(result, top_k : int, book_dict : dict):
    dataset = PredictDataset(result)
    loader = DataLoader(dataset, batch_size=2048, shuffle=True, drop_last=True)

    model.eval()
    scores = []
    with torch.no_grad():
        for X in loader:
            X = X.to(device)
            outputs = model(X)
            scores.append(outputs.cpu())
    scores = np.array(scores)
    scores = scores.flatten()
    scores = torch.tensor(scores)

    top_indices = torch.topk(scores, top_k).indices

    top_items = [items_to_predict.iloc[int(i)] for i in top_indices]

    for idx in range(len(top_items)):
        book_id = top_items[idx]['Id']
        book = book_dict[book_id]
        print(f" Recommended Book: {book}")



In [81]:
result = reco_user(212393)
pred_user(result, 5, book_dict)

            Id                                              Title  User_id  \
0       127550                     Its Only Art If Its Well Hung!   974357   
1        72368                           Dr. Seuss: American Icon   538929   
2        72745              Wonderful Worship in Smaller Churches  1001608   
3        37578                      Whispers of the Wicked Saints   725367   
4        11808  Nation Dance: Religion, Identity and Cultural ...   914561   
...        ...                                                ...      ...   
215827   15587                 Managing With Operational Research   186838   
215828  167056           Old Quebec,: The fortress of New France,   555120   
215829   53403  On the Life of Christ: Chanted Sermons by the ...   574109   
215830  205200                                        Ghost Story   248963   
215831   11892  An End to Welfare Rights: The Rediscovery of I...   307890   

        authors  categories  ratingsCount  
0         64441    